In [ ]:
# System Libraries
import importlib, sys, os
import numpy as np
import random

# TensorFlow Libraries
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
from tensorflow import keras
import tensorflow as tf

# User Libraries
sys.path.append("../")
from modules import common, chapter_01, chapter_09

# Reload Libraries
importlib.reload(chapter_01)
importlib.reload(chapter_09)
importlib.reload(common)

# Check GPU
print(tf.config.list_physical_devices())

# Module variables
batch_size = 64
epochs = 10

**Semantic Segmentation**

In [ ]:
augmentation = keras.Sequential(
    [
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomZoom(0.2),
        keras.layers.RandomRotation(0.2),
    ]
)

In [ ]:
i = keras.Input(shape=(200, 200, 3))
# ---------------------
# Augmentation
# ---------------------
x = augmentation(i)
# ---------------------
# Downsample
# ---------------------
x = keras.layers.Rescaling(1.0 / 255)(x)
x = keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation=tf.nn.relu, strides=2)(x)
x = keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.Conv2D(filters=128, kernel_size=3, padding="same", activation=tf.nn.relu, strides=2)(x)
x = keras.layers.Conv2D(filters=128, kernel_size=3, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.Conv2D(filters=256, kernel_size=3, padding="same", activation=tf.nn.relu, strides=2)(x)
x = keras.layers.Conv2D(filters=256, kernel_size=3, padding="same", activation=tf.nn.relu)(x)
# ---------------------
# Upsample
# ---------------------
x = keras.layers.Conv2DTranspose(filters=256, kernel_size=3, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.Conv2DTranspose(filters=256, kernel_size=3, padding="same", activation=tf.nn.relu, strides=2)(x)
x = keras.layers.Conv2DTranspose(filters=128, kernel_size=3, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.Conv2DTranspose(filters=128, kernel_size=3, padding="same", activation=tf.nn.relu, strides=2)(x)
x = keras.layers.Conv2DTranspose(filters=64, kernel_size=3, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.Conv2DTranspose(filters=64, kernel_size=3, padding="same", activation=tf.nn.relu, strides=2)(x)
# ---------------------
# Output
# ---------------------
o = keras.layers.Conv2D(filters=3, kernel_size=3, padding="same", activation=tf.nn.softmax)(x)
model = keras.Model(inputs=i, outputs=o)

In [6]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.legacy.RMSprop(0.001),
)

**Residual Connections without Maxpooling**

In [17]:
# ----------------------------
# Without residual connection
# ----------------------------
i = keras.Input(shape=(200, 200, 3))
x = keras.layers.Rescaling(1.0 / 255)(i)
x = keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation=tf.nn.relu)(x)
o = keras.layers.Conv2D(filters=3, kernel_size=3, padding="same", activation=tf.nn.softmax)(x)
# ----------------------------
# Without residual connection
# ----------------------------
i = keras.Input(shape=(200, 200, 3))
x = keras.layers.Rescaling(1.0 / 255)(i)
x = keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation=tf.nn.relu)(x)
residual = x
x = keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation=tf.nn.relu)(x)
residual = keras.layers.Conv2D(filters=64, kernel_size=1, padding="same", activation=None, strides=1)(residual)
x = keras.layers.add([x, residual])
o = keras.layers.Conv2D(filters=3, kernel_size=3, padding="same", activation=tf.nn.softmax)(x)

**Residual Connections with Maxpooling**

In [25]:
# ----------------------------
# Without residual connection
# ----------------------------
i = keras.Input(shape=(200, 200, 3))
x = keras.layers.Rescaling(1.0 / 255)(i)
x = keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.MaxPool2D(pool_size=2)(x)
x = keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.MaxPool2D(pool_size=2)(x)
o = keras.layers.Conv2D(filters=3, kernel_size=3, padding="same", activation=tf.nn.softmax)(x)
model1 = keras.Model(inputs=i, outputs=o)
# ----------------------------
# Without residual connection
# ----------------------------
i = keras.Input(shape=(200, 200, 3))
x = keras.layers.Rescaling(1.0 / 255)(i)
x = keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.MaxPool2D(pool_size=2)(x)
x = keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation=tf.nn.relu)(x)
residual = x
x = keras.layers.MaxPool2D(pool_size=2)(x)
residual = keras.layers.Conv2D(filters=64, kernel_size=1, padding="same", activation=None, strides=2)(residual)
x = keras.layers.add([residual, x])
o = keras.layers.Conv2D(filters=3, kernel_size=3, padding="same", activation=tf.nn.softmax)(x)
model2 = keras.Model(inputs=i, outputs=o)

In [26]:
model1.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_31 (InputLayer)       [(None, 200, 200, 3)]     0         
                                                                 
 rescaling_29 (Rescaling)    (None, 200, 200, 3)       0         
                                                                 
 conv2d_91 (Conv2D)          (None, 200, 200, 32)      896       
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 100, 100, 32)      0         
 ng2D)                                                           
                                                                 
 conv2d_92 (Conv2D)          (None, 100, 100, 64)      18496     
                                                                 
 max_pooling2d_19 (MaxPooli  (None, 50, 50, 64)        0         
 ng2D)                                                    

In [27]:
model2.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_32 (InputLayer)       [(None, 200, 200, 3)]        0         []                            
                                                                                                  
 rescaling_30 (Rescaling)    (None, 200, 200, 3)          0         ['input_32[0][0]']            
                                                                                                  
 conv2d_94 (Conv2D)          (None, 200, 200, 32)         896       ['rescaling_30[0][0]']        
                                                                                                  
 max_pooling2d_20 (MaxPooli  (None, 100, 100, 32)         0         ['conv2d_94[0][0]']           
 ng2D)                                                                                     

**Batch normalization**

In [31]:
# ----------------------------
# Without residual connection
# ----------------------------
i = keras.Input(shape=(200, 200, 3))
x = keras.layers.Rescaling(1.0 / 255)(i)
x = keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation=tf.nn.relu)(x)
o = keras.layers.Conv2D(filters=3, kernel_size=3, padding="same", activation=tf.nn.softmax)(x)
# ----------------------------
# Without residual connection
# ----------------------------
i = keras.Input(shape=(200, 200, 3))
x = keras.layers.Rescaling(1.0 / 255)(i)
x = keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", use_bias=False, activation=None)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation(activation=tf.nn.relu)(x)
x = keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", use_bias=False, activation=None)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation(activation=tf.nn.relu)(x)
o = keras.layers.Conv2D(filters=3, kernel_size=3, padding="same", activation=tf.nn.softmax)(x)

**Depthwise separable convolution**

In [33]:
# ----------------------------
# With standard convolution
# ----------------------------
i = keras.Input(shape=(200, 200, 3))
x = keras.layers.Rescaling(1.0 / 255)(i)
x = keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation=tf.nn.relu)(x)
o = keras.layers.Conv2D(filters=3, kernel_size=3, padding="same", activation=tf.nn.softmax)(x)
model1 = keras.Model(inputs=i, outputs=o)
# ----------------------------
# With separable convolution
# ----------------------------
i = keras.Input(shape=(200, 200, 3))
x = keras.layers.Rescaling(1.0 / 255)(i)
x = keras.layers.SeparableConv2D(filters=32, kernel_size=3, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.SeparableConv2D(filters=64, kernel_size=3, padding="same", activation=tf.nn.relu)(x)
o = keras.layers.SeparableConv2D(filters=3, kernel_size=3, padding="same", activation=tf.nn.softmax)(x)
model2 = keras.Model(inputs=i, outputs=o)

In [34]:
model1.summary()

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_42 (InputLayer)       [(None, 200, 200, 3)]     0         
                                                                 
 rescaling_40 (Rescaling)    (None, 200, 200, 3)       0         
                                                                 
 conv2d_121 (Conv2D)         (None, 200, 200, 32)      896       
                                                                 
 conv2d_122 (Conv2D)         (None, 200, 200, 64)      18496     
                                                                 
 conv2d_123 (Conv2D)         (None, 200, 200, 3)       1731      
                                                                 
Total params: 21123 (82.51 KB)
Trainable params: 21123 (82.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
model2.summary()

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_43 (InputLayer)       [(None, 200, 200, 3)]     0         
                                                                 
 rescaling_41 (Rescaling)    (None, 200, 200, 3)       0         
                                                                 
 separable_conv2d_3 (Separa  (None, 200, 200, 32)      155       
 bleConv2D)                                                      
                                                                 
 separable_conv2d_4 (Separa  (None, 200, 200, 64)      2400      
 bleConv2D)                                                      
                                                                 
 separable_conv2d_5 (Separa  (None, 200, 200, 3)       771       
 bleConv2D)                                                      
                                                          